In [2]:
!unzip /content/Fake.csv.zip

Archive:  /content/Fake.csv.zip
  inflating: Fake.csv                


In [1]:
!unzip /content/True.csv.zip

Archive:  /content/True.csv.zip
  inflating: True.csv                


In [ ]:
!pip install datasets evaluate
!pip install accelerate -U

In [2]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as F
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification
import torch
import csv
import pandas as pd
from transformers import Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding
import evaluate

In [3]:
def merge_data(data_true, data_false, dest):
    df1 = pd.read_csv(data_true)
    df1['label'] = np.ones(len(df1), dtype=int)
    df2 = pd.read_csv(data_false)
    df2['label'] = np.zeros(len(df2), dtype=int)
    merged_df = pd.concat([df1, df2], axis=0)

    random_indices = np.random.permutation(merged_df.index)

    # Shuffle the DataFrame using the random indices
    shuffled_df = merged_df.loc[random_indices].reset_index(drop=True)
    shuffled_df.to_csv(dest, index=False)



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# paths for data files
data_true = '/content/True.csv'
data_false = '/content/Fake.csv'
data_merge = '/content/merged.csv'

# merge data files
merge_data(data_true, data_false, data_merge)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
id2label = {0: "Fake", 1: "Real"}
label2id = {"Fake": 0, "Real": 1}
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training with HuggingFace Trainer

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
df = pd.read_csv(data_merge)
ds = Dataset.from_pandas(df)
ds = ds.remove_columns(['title', 'subject', 'date'])

In [ ]:
ds.push_to_hub('RealFake', private=True)

In [25]:
def tokenizer_function(example):
    return tokenizer(example['text'], truncation=True)

ds = ds.map(tokenizer_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest')

Map:   0%|          | 0/87732 [00:00<?, ? examples/s]

In [26]:
dss = ds.train_test_split(0.3)

In [30]:
id2label = {0: "Fake", 1: "Real"}
label2id = {"Fake": 0, "Real": 1}
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
metric = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    results = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": results["accuracy"]}



In [ ]:
training_args = TrainingArguments(
    output_dir="Real_fake_bert",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dss['train'],
    eval_dataset=dss['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
0,0.001300,0.000826,0.999924
